In [1]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#creation of dataset
import lib.ml_workflow.create_dataset as cds
from lib.raman_lib.misc import load_data

#quality control
import lib.ml_workflow.quality_control as qc
from lib.raman_lib.preprocessing import RangeLimiter
from lib.raman_lib.visualization import plot_spectra_peaks
from lib.raman_lib.spectra_scoring import score_names

#preprocessing
from lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import LeaveOneGroupOut, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [3]:
FILE_PREFIX = "HL_428_E_R"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"


## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [4]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [5]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [6]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [7]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [8]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [9]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [10]:
SCORING = ['roc_auc', 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [11]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_etoposide, path_resveratrol], ['Etoposide', 'Resveratrol'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [12]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [13]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement GroupKFold CV

In [14]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
logo = LeaveOneGroupOut()

## Baseline with LDA alone

In [15]:
clf = LinearDiscriminantAnalysis()
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.28488207, 0.16191006, 0.20940709]),
 'score_time': array([0.00601816, 0.00279403, 0.00411797]),
 'test_roc_auc': array([0.73047786, 0.72267298, 0.6881299 ]),
 'test_accuracy': array([0.67487685, 0.71957672, 0.67307692]),
 'test_f1': array([0.7027027 , 0.70056497, 0.69642857])}

In [16]:
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.29184675, 0.31818581, 0.393646  ]),
 'score_time': array([0.0228343 , 0.02094316, 0.01687407]),
 'test_roc_auc': array([0.44531857, 0.47025017, 0.58930571]),
 'test_accuracy': array([0.46305419, 0.49206349, 0.61057692]),
 'test_f1': array([0.42931937, 0.47826087, 0.64317181])}

In [17]:
clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/Users/lukas/opt/anaconda3/envs/internship/lib/python3.9/site-packages/joblib/memory.py", line 349, in __ca

In [ ]:
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns)-1), 1) +
                                                            np.diag(np.ones(len(wns)-1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)


In [ ]:
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)


In [ ]:
param_grid = {"peaks__min_dist": range(
    10, 151, 50
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING).fit(X,y, groups=groups)